In [1]:
!git clone -b 3-dialogpt-training https://github.com/wtaisner/atla-generator.git #fix that later to main (when merge)

Cloning into 'atla-generator'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 81 (delta 28), reused 53 (delta 12), pack-reused 0
Unpacking objects: 100% (81/81), done.


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 57.4 MB/s 
     |████████████████████████████████| 596 kB 60.2 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import importlib.util       
 
import_data = importlib.util.spec_from_file_location(
  "mod", "/content/atla-generator/src/data.py")   
data = importlib.util.module_from_spec(import_data)       
import_data.loader.exec_module(data)

import_conversation = importlib.util.spec_from_file_location(
  "mod", "/content/atla-generator/src/DialoGPT/conversation.py")   
conversation = importlib.util.module_from_spec(import_conversation)       
import_conversation.loader.exec_module(conversation)

import_prepare_dataset = importlib.util.spec_from_file_location(
  "mod", "/content/atla-generator/src/DialoGPT/prepare_dataset.py")   
prepare_dataset = importlib.util.module_from_spec(import_prepare_dataset)       
import_prepare_dataset.loader.exec_module(prepare_dataset)

Add zip with data (if needed change the name - default is archive.zip)

In [4]:
!unzip archive.zip -d '/content/atla-generator/data'

Archive:  archive.zip
  inflating: /content/atla-generator/data/avatar.csv  
  inflating: /content/atla-generator/data/avatar_data.csv  
  inflating: /content/atla-generator/data/scenes.csv  
  inflating: /content/atla-generator/data/series_names.csv  


In [5]:
import torch
from torch import cuda
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Set parameters

In [6]:
SEED = 42
MODEL = "microsoft/DialoGPT-small"

NAME = "Iroh"
N = 5

TRAIN_SIZE = 0.8

TRAIN_BATCH = 4
EVAL_BATCH = 4
EPOCHS = 10
OUTPUT_DIR = "/content/atla-generator/outputs/DialoGPT"
OVERWRITE_OUTPUT_DIR = True
EVAL_STRATEGY="epoch"
LEARNING_RATE = 5e-5
LOAD_BEST_MODEL_AT_THE_END = True
PREDICTION_LOSS_ONLY = True

STEPS = 7

In [7]:
torch.manual_seed(SEED)
device = 'cuda' if cuda.is_available() else 'cpu'

# Load model and tokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.pad_token = 0

model = AutoModelForCausalLM.from_pretrained(MODEL)
model.to(device)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

# Read and reformat data, create train and eval split

In [9]:
train_size = TRAIN_SIZE

df = data.read_dataframe(path='/content/atla-generator/data/avatar.csv')
df = prepare_dataset.create_context(df, name=NAME, n=N)

train_dataset = df.sample(frac=train_size, random_state=SEED)
eval_dataset = df.drop(train_dataset.index).reset_index(drop = True)
train_dataset = train_dataset.reset_index(drop = True)

print(f"df shape: {df.shape}")
print(f"train shape: {train_dataset.shape}")
print(f"eval shape: {eval_dataset.shape}")

df shape: (337, 6)
train shape: (270, 6)
eval shape: (67, 6)


In [10]:
train_set = prepare_dataset.ConversationDataset(train_dataset, tokenizer)
eval_set = prepare_dataset.ConversationDataset(eval_dataset, tokenizer)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Fine-tune model (training)

In [11]:
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=OVERWRITE_OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=TRAIN_BATCH,
    per_device_eval_batch_size=EVAL_BATCH,
    prediction_loss_only=PREDICTION_LOSS_ONLY,
    evaluation_strategy=EVAL_STRATEGY,
    save_strategy=EVAL_STRATEGY,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    load_best_model_at_end=LOAD_BEST_MODEL_AT_THE_END,
)

In [12]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_set,
    eval_dataset=eval_set,
    data_collator=data_collator,
)

In [13]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 270
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 680


Epoch,Training Loss,Validation Loss
1,No log,3.466888
2,No log,3.171084
3,No log,3.000600
4,No log,2.878943
5,No log,2.810711
6,No log,2.749407
7,No log,2.715672
8,2.799800,2.703493
9,2.799800,2.688128
10,2.799800,2.690668


***** Running Evaluation *****
  Num examples = 67
  Batch size = 4
Saving model checkpoint to /content/atla-generator/outputs/DialoGPT/checkpoint-68
Configuration saved in /content/atla-generator/outputs/DialoGPT/checkpoint-68/config.json
Model weights saved in /content/atla-generator/outputs/DialoGPT/checkpoint-68/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 67
  Batch size = 4
Saving model checkpoint to /content/atla-generator/outputs/DialoGPT/checkpoint-136
Configuration saved in /content/atla-generator/outputs/DialoGPT/checkpoint-136/config.json
Model weights saved in /content/atla-generator/outputs/DialoGPT/checkpoint-136/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 67
  Batch size = 4
Saving model checkpoint to /content/atla-generator/outputs/DialoGPT/checkpoint-204
Configuration saved in /content/atla-generator/outputs/DialoGPT/checkpoint-204/config.json
Model weights saved in /content/atla-generator/outputs/DialoGPT/checkpoint-204/pyto

TrainOutput(global_step=680, training_loss=2.549476578656365, metrics={'train_runtime': 209.4501, 'train_samples_per_second': 12.891, 'train_steps_per_second': 3.247, 'total_flos': 213518458368000.0, 'train_loss': 2.549476578656365, 'epoch': 10.0})

In [14]:
trainer.save_model(OUTPUT_DIR) #remeber to download the model

Saving model checkpoint to /content/atla-generator/outputs/DialoGPT
Configuration saved in /content/atla-generator/outputs/DialoGPT/config.json
Model weights saved in /content/atla-generator/outputs/DialoGPT/pytorch_model.bin


# Load saved model and chat

In [15]:
model = AutoModelForCausalLM.from_pretrained(OUTPUT_DIR)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

loading configuration file /content/atla-generator/outputs/DialoGPT/config.json
Model config GPT2Config {
  "_name_or_path": "/content/atla-generator/outputs/DialoGPT",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "conversational": {
      "max_length": 1000
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "use_cache": true,
 

In [16]:
conversation.chat_with_me(model, tokenizer, steps=STEPS)

>> User:how are you?
Bot: It's okay.
>> User:Do you know Zukko?
Bot: I know who he is.
>> User:How about some tea?
Bot: Of course!
>> User:I will capture the avatar
Bot: And you will serve as his assistant.
>> User:Why do you say so uncle, I want to impress my father
Bot: You do not want to know how I look in my new found identity.
>> User:What is your new found identity?
Bot: You are a beautiful young man.  You look nothing like your father.
>> User:Thanks, uncle Iroh
Bot: It is good to see you're enjoying the tea.
